In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.expand_frame_repr", False)


In [5]:
deliveries = pd.read_csv("deliveries.csv")
deliveries.rename(columns={"batter": "batsman"}, inplace=True)
matches = pd.read_csv("matches.csv")
deliveries.head(5)

,match_id,inning,batting_team,bowling_team,over,ball,batsman,bowler,non_striker,batsman_runs,extra_runs,total_runs,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder
0,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,1,SC Ganguly,P Kumar,BB McCullum,0,1,1,legbyes,0,NaN,NaN,NaN
1,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,2,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
2,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,3,BB McCullum,P Kumar,SC Ganguly,0,1,1,wides,0,NaN,NaN,NaN
3,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,4,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
4,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,5,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN


In [6]:
def balls_per_dismissal(balls,dismissals):
    if dismissals > 0:
        return balls/dismissals
    else:
        return balls/1
    
def balls_per_boundary(balls,boundaries):
    if boundaries > 0:
        return balls/boundaries
    else:
        return balls/1
    
def phase(over):
    if over < 6:
        return "Powerplay"
    elif over < 15:
        return "Middle"
    else:
        return "Death"

In [20]:
def player_statistics(df):
    df["isDot"] = df["batsman_runs"].apply(lambda x: 1 if x == 0 else 0)
    df["isOne"] = df["batsman_runs"].apply(lambda x: 1 if x == 1 else 0)
    df["isTwo"] = df["batsman_runs"].apply(lambda x: 1 if x == 2 else 0)
    df["isThree"] = df["batsman_runs"].apply(lambda x: 1 if x == 3 else 0)
    df["isFour"] = df["batsman_runs"].apply(lambda x: 1 if x == 4 else 0)
    df["isFive"] = df["batsman_runs"].apply(lambda x: 1 if x == 5 else 0)
    df["isSix"] = df["batsman_runs"].apply(lambda x: 1 if x == 6 else 0)
    
    dots = pd.DataFrame(df.groupby(['batsman'])['isDot'].sum()).reset_index().rename(columns = {'isDot':'Dots'})
    ones = pd.DataFrame(df.groupby(['batsman'])['isOne'].sum()).reset_index().rename(columns = {'isOne':'Ones'})
    twos = pd.DataFrame(df.groupby(['batsman'])['isTwo'].sum()).reset_index().rename(columns = {'isTwo':'Twos'})
    threes = pd.DataFrame(df.groupby(['batsman'])['isThree'].sum()).reset_index().rename(columns = {'isThree':'Threes'})
    sixes = pd.DataFrame(df.groupby(['batsman'])['isSix'].sum()).reset_index().rename(columns = {'isSix':'Sixes'})
    fours = pd.DataFrame(df.groupby(['batsman'])['isFour'].sum()).reset_index().rename(columns = {'isFour':'Fours'})
    fives = pd.DataFrame(df.groupby(['batsman'])['isFive'].sum()).reset_index().rename(columns = {'isFive':'Fives'})

    dismissals = pd.DataFrame(df.groupby(["batsman"])["player_dismissed"].count().reset_index().rename(columns = {'player_dismissed':'dismissals'}))
    runs = pd.DataFrame(df.groupby(["batsman"])["batsman_runs"].sum()).reset_index().rename(columns = {"batsman_runs":"runs"})
    balls = pd.DataFrame(df.groupby(['batsman'])['match_id'].count()).reset_index().rename(columns = {'match_id':'ball'})
    innings= pd.DataFrame(df.groupby(['batsman'])['match_id'].apply(lambda x: len(list(np.unique(x))))).reset_index().rename(columns = {'match_id':'inning'})
    
    df= pd.merge(runs, innings, on ='batsman').merge(balls, on ='batsman').merge(dismissals, on ='batsman').merge(fours, on ='batsman').merge(sixes, on ='batsman').merge(dots, on ='batsman').merge(ones, on ='batsman').merge(twos, on ='batsman').merge(threes, on ='batsman')
    
    df["RPI"] = df.apply(lambda x: (x["runs"]/x["inning"]),axis = 1)
    df["BPD"] = df.apply(lambda x: balls_per_dismissal(x["ball"],x["dismissals"]),axis = 1)
    df["BPB"] = df.apply(lambda x: balls_per_dismissal(x["ball"],x["Fours"]+x["Sixes"]),axis = 1)
    df["SR"] = df.apply(lambda x: 100*(x["runs"]/x["ball"]),axis = 1)
 
    return df

In [21]:
df = player_statistics(deliveries)

In [22]:
df.head(5)

,batsman,runs,inning,ball,dismissals,Fours,Sixes,Dots,Ones,Twos,Threes,RPI,BPD,BPB,SR
0,A Ashish Reddy,280,23,196,15,16,15,61,83,20,1,12.173913,13.066667,6.322581,142.857143
1,A Badoni,634,35,505,26,46,24,183,201,48,3,18.114286,19.423077,7.214286,125.544554
2,A Chandila,4,2,7,1,0,0,3,4,0,0,2.000000,7.000000,7.000000,57.142857
3,A Chopra,53,6,75,5,7,0,45,21,2,0,8.833333,15.000000,10.714286,70.666667
4,A Choudhary,25,3,20,2,1,1,4,13,1,0,8.333333,10.000000,10.000000,125.000000


In [24]:
deliveries["phase"] = deliveries["over"].apply(lambda x: phase(x))

In [30]:
def phasewise_player_statistics(df,phase):
    
    df = df[df.phase == phase]
    df.reset_index(inplace = True, drop = True)
    
    df["isDot"] = df["batsman_runs"].apply(lambda x: 1 if x == 0 else 0)
    df["isOne"] = df["batsman_runs"].apply(lambda x: 1 if x == 1 else 0)
    df["isTwo"] = df["batsman_runs"].apply(lambda x: 1 if x == 2 else 0)
    df["isThree"] = df["batsman_runs"].apply(lambda x: 1 if x == 3 else 0)
    df["isFour"] = df["batsman_runs"].apply(lambda x: 1 if x == 4 else 0)
    df["isFive"] = df["batsman_runs"].apply(lambda x: 1 if x == 5 else 0)
    df["isSix"] = df["batsman_runs"].apply(lambda x: 1 if x == 6 else 0)
    
    dots = pd.DataFrame(df.groupby(['batsman'])['isDot'].sum()).reset_index().rename(columns = {'isDot':'Dots'})
    ones = pd.DataFrame(df.groupby(['batsman'])['isOne'].sum()).reset_index().rename(columns = {'isOne':'Ones'})
    twos = pd.DataFrame(df.groupby(['batsman'])['isTwo'].sum()).reset_index().rename(columns = {'isTwo':'Twos'})
    threes = pd.DataFrame(df.groupby(['batsman'])['isThree'].sum()).reset_index().rename(columns = {'isThree':'Threes'})
    sixes = pd.DataFrame(df.groupby(['batsman'])['isSix'].sum()).reset_index().rename(columns = {'isSix':'Sixes'})
    fours = pd.DataFrame(df.groupby(['batsman'])['isFour'].sum()).reset_index().rename(columns = {'isFour':'Fours'})
    fives = pd.DataFrame(df.groupby(['batsman'])['isFive'].sum()).reset_index().rename(columns = {'isFive':'Fives'})

    dismissals = pd.DataFrame(df.groupby(["batsman"])["player_dismissed"].count().reset_index().rename(columns = {'player_dismissed':'dismissals'}))
    runs = pd.DataFrame(df.groupby(["batsman"])["batsman_runs"].sum()).reset_index().rename(columns = {"batsman_runs":"runs"})
    balls = pd.DataFrame(df.groupby(['batsman'])['match_id'].count()).reset_index().rename(columns = {'match_id':'ball'})
    innings= pd.DataFrame(df.groupby(['batsman'])['match_id'].apply(lambda x: len(list(np.unique(x))))).reset_index().rename(columns = {'match_id':'inning'})
    
    df= pd.merge(innings, runs, on ='batsman').merge(balls, on ='batsman').merge(dismissals, on ='batsman').merge(fours, on ='batsman').merge(fives, on = "batsman").merge(sixes, on ='batsman').merge(dots, on ='batsman').merge(ones, on ='batsman').merge(twos, on ='batsman').merge(threes, on ='batsman')
    
    df["RPI"] = df.apply(lambda x: (x["runs"]/x["inning"]),axis = 1)
    df["BPD"] = df.apply(lambda x: balls_per_dismissal(x["ball"],x["dismissals"]),axis = 1)
    df["BPB"] = df.apply(lambda x: balls_per_dismissal(x["ball"],x["Fours"]+x["Sixes"]),axis = 1)
    df["SR"] = df.apply(lambda x: 100*(x["runs"]/x["ball"]),axis = 1)
 
    return df

In [31]:
df = phasewise_player_statistics(deliveries,"Powerplay")

In [32]:
df.head()

,batsman,inning,runs,ball,dismissals,Fours,Fives,Sixes,Dots,Ones,Twos,Threes,RPI,BPD,BPB,SR
0,A Ashish Reddy,1,5,7,1,1,0,0,5,1,0,0,5.000000,7.000000,7.00,71.428571
1,A Badoni,7,18,38,3,2,0,0,27,8,1,0,2.571429,12.666667,19.00,47.368421
2,A Chopra,5,27,41,2,4,0,0,26,11,0,0,5.400000,20.500000,10.25,65.853659
3,A Flintoff,1,14,15,0,1,0,1,11,1,0,1,14.000000,15.000000,7.50,93.333333
4,A Manohar,2,12,18,1,2,0,0,12,4,0,0,6.000000,18.000000,9.00,66.666667


In [33]:
wt_sr, wt_rpi, wt_bpd, wt_bpb = 0.38, 0.25, 0.12, 0.26

In [35]:
pp_df  = df[df.inning > 20]
pp_df.reset_index(drop = True, inplace = True)
pp_df.head()

,batsman,inning,runs,ball,dismissals,Fours,Fives,Sixes,Dots,Ones,Twos,Threes,RPI,BPD,BPB,SR
0,AB de Villiers,81,741,630,22,100,0,20,319,164,24,3,9.148148,28.636364,5.250000,117.619048
1,AC Gilchrist,80,1385,1080,48,191,0,49,556,244,37,3,17.312500,22.500000,4.500000,128.240741
2,AJ Finch,71,911,826,40,120,0,25,445,194,39,3,12.830986,20.650000,5.696552,110.290557
3,AM Rahane,157,2319,2013,72,302,0,49,959,603,86,14,14.770701,27.958333,5.735043,115.201192
4,AT Rayudu,82,777,750,23,93,0,24,395,215,23,0,9.475610,32.608696,6.410256,103.600000


In [36]:
pp_df["calc_SR"] = pp_df["SR"].apply(lambda x: x*x)
pp_df["calc_BPD"] = pp_df["BPD"].apply(lambda x: x*x)
pp_df["calc_BPB"] = pp_df["BPB"].apply(lambda x: x*x)
pp_df["calc_RPI"] = pp_df["RPI"].apply(lambda x: x*x)

sqrt_sr, sqrt_rpi, sqrt_bpd, sqrt_bpb = np.sqrt(pp_df[["calc_SR","calc_RPI","calc_BPD","calc_BPB"]].sum(axis = 0))

pp_df["calc_SR"] = pp_df["calc_SR"].apply(lambda x: x/sqrt_sr)
pp_df["calc_BPD"] = pp_df["calc_BPD"].apply(lambda x: x/sqrt_bpd)
pp_df["calc_BPB"] = pp_df["calc_BPB"].apply(lambda x: x/sqrt_bpb)
pp_df["calc_RPI"] = pp_df["calc_RPI"].apply(lambda x: x/sqrt_rpi)

pp_df["calc_SR"] = pp_df["calc_SR"].apply(lambda x: x*wt_sr)
pp_df["calc_BPD"] = pp_df["calc_BPD"].apply(lambda x: x*wt_bpd)
pp_df["calc_BPB"] = pp_df["calc_BPB"].apply(lambda x: x*wt_bpb)
pp_df["calc_RPI"] = pp_df["calc_RPI"].apply(lambda x: x*wt_rpi)

best_sr,worst_sr = max(pp_df["calc_SR"]), min(pp_df["calc_SR"])
best_bpd,worst_bpd = max(pp_df["calc_BPD"]), min(pp_df["calc_BPD"])
best_bpb,worst_bpb = max(pp_df["calc_BPB"]), min(pp_df["calc_BPB"])
best_rpi,worst_rpi = max(pp_df["calc_RPI"]), min(pp_df["calc_RPI"])

In [37]:
pp_df['dev_best_SR'] = pp_df['calc_SR'].apply(lambda x: (x-best_sr)*(x-best_sr)) 
pp_df['dev_best_RPI'] = pp_df['calc_RPI'].apply(lambda x: (x-best_rpi)*(x-best_rpi)) 
pp_df['dev_best_BPD'] = pp_df['calc_BPD'].apply(lambda x: (x-best_bpd)*(x-best_bpd)) 
pp_df['dev_best_BPB'] = pp_df['calc_BPB'].apply(lambda x: (x-best_bpb)*(x-best_bpb))

pp_df['dev_best_sqrt'] = pp_df.apply(lambda x: x['dev_best_SR'] + x['dev_best_RPI'] + x['dev_best_BPD'] + x['dev_best_BPB'], axis = 1)

pp_df['dev_worst_SR'] = pp_df['calc_SR'].apply(lambda x: (x-worst_sr)*(x-worst_sr)) 
pp_df['dev_worst_RPI'] = pp_df['calc_RPI'].apply(lambda x: (x-worst_rpi)*(x-worst_rpi)) 
pp_df['dev_worst_BPD'] = pp_df['calc_BPD'].apply(lambda x: (x-worst_bpd)*(x-worst_bpd)) 
pp_df['dev_worst_BPB'] = pp_df['calc_BPB'].apply(lambda x: (x-worst_bpb)*(x-worst_bpb))

pp_df['dev_worst_sqrt'] = pp_df.apply(lambda x: x['dev_worst_SR'] + x['dev_worst_RPI'] + x['dev_worst_BPD'] + x['dev_worst_BPB'], axis = 1)

In [38]:
pp_df["Score"] = pp_df.apply(lambda x: x["dev_worst_sqrt"]/(x["dev_worst_sqrt"]+x["dev_best_sqrt"]),axis = 1)

In [39]:
pp_df[['batsman', 'Score']].head()

,batsman,Score
0,AB de Villiers,0.275603
1,AC Gilchrist,0.495530
2,AJ Finch,0.162761
3,AM Rahane,0.239523
4,AT Rayudu,0.096657


In [44]:
top10 = pd.DataFrame(pp_df[[ 'batsman','inning', 'runs', 'ball', 'dismissals', 'Fours', 'Sixes', 'SR', 'BPB', 'Score']].sort_values(['Score'], ascending = False).reset_index(drop = True).head(10))

In [45]:
top10

,batsman,inning,runs,ball,dismissals,Fours,Sixes,SR,BPB,Score
0,SP Narine,58,946,577,41,119,55,163.951473,3.316092,0.958091
1,Abhishek Sharma,40,758,501,22,90,35,151.297405,4.008000,0.905248
2,YBK Jaiswal,52,1009,677,25,149,36,149.039882,3.659459,0.887696
3,JM Bairstow,47,864,606,22,113,34,142.574257,4.122449,0.797742
4,P Simran Singh,31,499,350,20,63,23,142.571429,4.069767,0.782586
5,PP Shaw,79,1347,946,51,193,37,142.389006,4.113043,0.781635
6,CA Lynn,40,779,553,17,99,37,140.867993,4.066176,0.776967
7,E Lewis,22,420,298,13,49,21,140.939597,4.257143,0.763632
8,YK Pathan,33,361,256,12,47,16,141.015625,4.063492,0.753948
9,SA Yadav,64,859,628,17,120,27,136.783439,4.272109,0.698633
